In [82]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [83]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [ ]:
print("msno")
msno_values = data.msno.unique()
print("end msno")
del(data)

msno
end msno


In [ ]:
user_logs = pd.read_csv('raw_data/user_logs.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
user_logs_2 = pd.read_csv('raw_data/user_logs_v2.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
#user_logs = pd.concat((user_logs, user_logs_2))

In [ ]:
user_logs.head()

In [ ]:
counts = user_logs["msno"].value_counts()
counts.to_csv("cleaned_input/counts.csv")

del(user_logs)
del(user_logs_2)

In [ ]:
#get number of unique visits
user_features = pd.DataFrame() 

user_features['msno'] = np.nan #counts.index
user_features["frequency"] = np.nan #counts.values
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

In [ ]:
#read in user_logs
import time
chunksize = 10000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    print(i)
    start_time = time.time()
    chunk = chunk[chunk['msno'].isin(msno_values)]
    rows = chunk.msno.unique()
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    print("processing ",i,"took %s seconds" % (time.time() - start_time))
    n = 0 
    print(len(rows))
    for row in rows:
        row_time = time.time()
        df = chunk[chunk["msno"] == row]
        differences = df.date.diff()
        gap_days = differences.sum()
        average_gap_days = differences.mean()
        if len(differences.nlargest(1)) > 0:
            longest_gap = differences.nlargest(1).iloc[0].total_seconds()
        else:
            longest_gap = -1
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        user_features = user_features.append([row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29])
        n += 1
        print("row ",i,"took %s seconds" % (time.time() - row_time))
    del(chunk)
    i += 1 
    print("Round ",i,"took %s seconds" % (time.time() - start_time))
user_features.head()

In [ ]:
user_features.to_csv("user_logs_features.csv")

In [ ]:
#read in user_logs_2
chunksize = 1000000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs_v2.csv', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    print(i)
    rows = chunk.msno.unique()#find unique chunks
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    for row in rows:
        df = chunk[chunk["msno"] == row]
        differences = df.date.diff()
        gap_days = differences.sum()
        average_gap_days = differences.mean()
        longest_gap = differences.nlargest(1)
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        user_features[user_features['msno'] == row]["gap_days"] = gap_days
        user_features[user_features['msno'] == row]["average_gap_days"] = average_gap_days
        user_features[user_features['msno'] == row]["longest_gap"] = longest_gap
        user_features[user_features['msno'] == row]["gaps_longer_29"] = gaps_longer_29
    del(chunk)
    i += 1 
user_features.head()

In [ ]:
#combine duplicates? 

In [ ]:
user_features.head()

In [ ]:
user_features.to_csv("user_logs_features.csv")

In [ ]:
#combine what you 